In [23]:
# from datasets import load_dataset

# dataset = load_dataset("yelp_review_full")

In [1]:
import pickle
# with open('yelp_dataset.pkl', 'wb') as file:
#     pickle.dump(dataset, file)

In [2]:
with open('yelp_dataset.pkl', 'rb') as file:
    dataset = pickle.load(file)

/home/sriteja/anaconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [26]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 50000
    })
})


In [3]:
train_dataset = dataset['train']
test_dataset = dataset['test']

# Convert train and test datasets to arrays
train_data = train_dataset['text']
train_labels = train_dataset['label']
test_data = test_dataset['text']
test_labels = test_dataset['label']

# Convert labels to lists (optional)
# train_labels = train_labels.tolist()
# test_labels = test_labels.tolist()

In [28]:
print(train_data[0])
print(train_labels[0])


dr. goldberg offers everything i look for in a general practitioner.  he's nice and easy to talk to without being patronizing; he's always on time in seeing his patients; he's affiliated with a top-notch hospital (nyu) which my parents have explained to me is very important in case something happens and you need surgery; and you can get referrals to see specialists without having to see him first.  really, what more do you need?  i'm sitting here trying to think of any complaints i have about him, but i'm really drawing a blank.
4


## Preprocess

In [4]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/sriteja/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
import sys  
!{sys.executable} -m pip install contractions
import contractions

In [31]:
# use tokenizer to remove punctuation

def tokenize_text(text):
    expanded_words = []   
    for word in text.split():
        # using contractions.fix to expand the shortened words
        expanded_words.append(contractions.fix(word))   
    
    expanded_text = ' '.join(expanded_words)
    # print(expanded_text)
    
    text = expanded_text    
    
    # replace a-b with a and b
    text = text.replace('-', ' ')
    
    tokens = nltk.word_tokenize(text)
    # convert to lower case
    tokens = [w.lower() for w in tokens]
    # dr. = dr and st. = st and so on
    tokens = [w.replace('.', '') for w in tokens]
    
    # remove punctuation
    tokens = [word for word in tokens if word.isalpha()]
    
    # print(tokens)
    return tokens

# print(train_data[0])
# tokenize_text(train_data[0])


In [33]:
tokens = []
print(len(train_data))

for i in range(len(train_data)):
    tokens.append(tokenize_text(train_data[i]))
    if i % 1000 == 0:
        print(i)

650000
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000


In [34]:
# save to pkl file
with open('yelp_train_tokens.pkl', 'wb') as file:
    pickle.dump(tokens, file)

In [35]:
tokens_test = []
print(len(test_data))

for i in range(len(test_data)):
    tokens_test.append(tokenize_text(test_data[i]))
    if i % 1000 == 0:
        print(i)

with open('yelp_test_tokens.pkl', 'wb') as file:
    pickle.dump(tokens_test, file)

50000
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000


In [6]:
import numpy as np

train_tokens = []
with open('yelp_train_tokens.pkl', 'rb') as file:
    train_tokens = pickle.load(file)

test_tokens = []
with open('yelp_test_tokens.pkl', 'rb') as file:
    test_tokens = pickle.load(file)
    
print("Loaded tokens")



Loaded tokens


In [7]:
# Build the BoW representation manually
# Create a vocabulary by collecting unique words from the training data
vocab = set()
for tokens in train_tokens:
    vocab.update(tokens)

# Create a dictionary to map words to indices in the vocabulary
vocab_dict = {}
for i, word in enumerate(vocab):
    vocab_dict[word] = i

print(len(vocab_dict))
# Initialize BoW matrices for training and testing data


205290


In [8]:
reduced_train_tokens = train_tokens[:20000]

# Build the BoW representation manually
# Create a vocabulary by collecting unique words from the training data
vocab_reduced = set()
for tokens in reduced_train_tokens:
    vocab_reduced.update(tokens)

# Create a dictionary to map words to indices in the vocabulary
vocab_dict_reduced = {}
for i, word in enumerate(vocab_reduced):
    vocab_dict_reduced[word] = i

print(len(vocab_dict_reduced))

train_bow = np.zeros((len(reduced_train_tokens), len(vocab_reduced)))

print("bow train")
# Convert text to BoW vectors
for i, tokens in enumerate(reduced_train_tokens):
    if i % 5000 == 0:
        print(i)
    for token in tokens:
        train_bow[i][vocab_dict_reduced[token]] += 1




36370
bow train
0
5000
10000
15000


In [9]:
reduced_test_tokens = test_tokens[:5000]
test_bow = np.zeros((len(reduced_test_tokens), len(vocab_reduced)))

print("bow test")

for i, tokens in enumerate(reduced_test_tokens):
    if i % 1000 == 0:
        print(i)
    for token in tokens:
        if token in vocab_dict_reduced:
            test_bow[i][vocab_dict_reduced[token]] += 1
        # if there is an unknown word, ignore it

bow test
0
1000
2000
3000
4000


In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm

# Initialize the Logistic Regression classifier
classifier = LogisticRegression()

# Fit the classifier to the training data with a progress bar
with tqdm(total=len(train_bow[:20000])) as pbar:
    classifier.fit(train_bow[:20000], train_labels[:20000])
    pbar.update(len(train_bow[:20000]))

# Predict the labels of the test data with a progress bar
with tqdm(total=len(test_bow[:5000])) as pbar:
    y_pred = classifier.predict(test_bow[:5000])
    pbar.update(len(test_bow[:5000]))

# Calculate accuracy and other classification metrics
print('Accuracy: {}'.format(accuracy_score(test_labels[:5000], y_pred)))
print(classification_report(test_labels[:5000], y_pred))

# save the classifier
with open('yelp_classifier_log_reg_01.pkl', 'wb') as file:
    pickle.dump(classifier, file)

/home/sriteja/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.5284
              precision    recall  f1-score   support

           0       0.69      0.71      0.70      1141
           1       0.47      0.44      0.46      1009
           2       0.43      0.48      0.45      1003
           3       0.46      0.36      0.40       986
           4       0.56      0.64      0.59       861

    accuracy                           0.53      5000
   macro avg       0.52      0.53      0.52      5000
weighted avg       0.52      0.53      0.52      5000

